In [12]:
import pandas as pd

from pymongo import MongoClient
from dotenv import load_dotenv
import os
import numpy as np
import json

load_dotenv()

MONGO_PWD = os.getenv("MONGO_PWD")

In [29]:
client2 = MongoClient("mongodb://localhost:27017/")
db2 = client2.companies 
companies = db2.companies.find()
data_companies = pd.DataFrame(companies)
data_companies.shape

(18801, 42)

In [30]:
data_companies_filt = data_companies[["_id","acquisition","acquisitions","category_code","created_at","crunchbase_url","deadpooled_year","description","founded_year","homepage_url","name","number_of_employees","offices","overview","products","total_money_raised"]]

In [13]:
records = json.loads(data_companies_filt.T.to_json()).values()
db.companiesfilt.insert_many(records)

In [5]:
def conections(user,host="mmartin-c1diq.gcp.mongodb.net/test?retryWrites=true&w=majority",pwd=MONGO_PWD,port="27017"):
    client = MongoClient("mongodb+srv://"+user+":"+pwd+"@"+host)
    db = client.geoffice
    return db

In [6]:
db = conections("mmartin")
data = db.companies.find()
df = pd.DataFrame(data)


In [8]:
df.shape

(1304, 16)

In [21]:
airports = pd.read_csv("json/airports.csv", header=None)
airports = airports[[1,2,3,6,7]]
airports.columns = ["Airport","City","Country","lat","lon"]
airports.shape
                    

(7698, 5)

In [22]:
def geopointAir(lat,lon): 
    return {"type": "Point", "coordinates": [lon,lat]}

In [24]:
airports["geo"] = airports.apply(lambda x: geopointAir(x["lat"], x["lon"]), axis = 1)

In [26]:
airports.head()

,Airport,City,Country,lat,lon,geo
0,Goroka Airport,Goroka,Papua New Guinea,-6.081690,145.391998,"{'type': 'Point', 'coordinates': [145.39199829..."
1,Madang Airport,Madang,Papua New Guinea,-5.207080,145.789001,"{'type': 'Point', 'coordinates': [145.78900146..."
2,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,-5.826790,144.296005,"{'type': 'Point', 'coordinates': [144.29600524..."
3,Nadzab Airport,Nadzab,Papua New Guinea,-6.569803,146.725977,"{'type': 'Point', 'coordinates': [146.725977, ..."
4,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,-9.443380,147.220001,"{'type': 'Point', 'coordinates': [147.22000122..."


In [27]:
recordsair = json.loads(airports.T.to_json()).values()
db.airports.insert_many(recordsair)

In [31]:
# Delete closed companies 
data_companies_filt=data_companies_filt[data_companies_filt.deadpooled_year.isnull()]

In [34]:
data_companies_filt.shape

(17872, 16)

In [32]:
offices=[]
for i in range(len(data_companies_filt)):
    offices.append([data_companies_filt.offices.iloc[i],data_companies_filt._id.iloc[i]])
office_list=[]

for office in offices:
    a=pd.io.json.json_normalize(office[0])
    a["_id"]=office[1]
    office_list.append(a)
df_office=pd.concat(office_list,sort=True)

# Delete offices without lat. or long.
df_office=df_office.dropna(subset=['latitude', 'longitude'])

#Delete offices without city
df_office["city_clean"]=df_office.city.apply(lambda x: str(x).replace(" City",""))
df_office=df_office.drop("city",axis=1)
df_office=df_office.rename(columns={"city_clean":"city"})

filter_void=df_office.city=="" 
filter_None=df_office.city=="None"
df_office=df_office[~(filter_void | filter_None)]

In [33]:
print(df_office.shape)
df_office.head()

(9832, 10)


,_id,address1,address2,country_code,description,latitude,longitude,state_code,zip_code,city
0,52cdef7c4bab8bd675297d92,"8536 National Blvd, Suite A",None,USA,None,34.026,-118.38,CA,90232,Culver
0,52cdef7c4bab8bd675297d90,"959 Skyway Road, Suite 200",None,USA,None,37.5069,-122.248,CA,94070,San Carlos
0,52cdef7c4bab8bd675297d91,9229 W. Sunset Blvd.,,USA,Headquarters,34.0904,-118.393,CA,90069,West Hollywood
0,52cdef7c4bab8bd675297d96,None,None,USA,None,37.4841,-122.169,CA,None,Menlo Park
0,52cdef7c4bab8bd675297d8d,135 Mississippi St,,USA,None,37.7647,-122.395,CA,94107,San Francisco


In [41]:
companies2=data_companies_filt.merge(df_office, on="_id")
companies2=companies2.drop(columns=["_id","offices"])
companies2

,acquisition,acquisitions,category_code,created_at,crunchbase_url,deadpooled_year,description_x,founded_year,homepage_url,name,...,total_money_raised,address1,address2,country_code,description_y,latitude,longitude,state_code,zip_code,city
0,"{'price_amount': 20000000, 'price_currency_cod...",[],games_video,Thu May 31 21:11:51 UTC 2007,http://www.crunchbase.com/company/flektor,NaN,None,NaN,http://www.flektor.com,Flektor,...,$0,"8536 National Blvd, Suite A",None,USA,None,34.026,-118.38,CA,90232,Culver
1,"{'price_amount': 625000000, 'price_currency_co...",[],web,Fri Jun 08 12:19:51 UTC 2007,http://www.crunchbase.com/company/postini,NaN,None,1999.0,http://postini.com,Postini,...,$0,"959 Skyway Road, Suite 200",None,USA,None,37.5069,-122.248,CA,94070,San Carlos
2,"{'price_amount': None, 'price_currency_code': ...",[],web,Thu May 31 19:52:34 UTC 2007,http://www.crunchbase.com/company/geni,NaN,Geneology social network site,2006.0,http://www.geni.com,Geni,...,$16.5M,9229 W. Sunset Blvd.,,USA,Headquarters,34.0904,-118.393,CA,90069,West Hollywood
3,"{'price_amount': None, 'price_currency_code': ...",[],web,Fri Jun 01 19:32:24 UTC 2007,http://www.crunchbase.com/company/gizmoz,NaN,Photo animation,2003.0,http://gizmoz.com,Gizmoz,...,$18.1M,None,None,USA,None,37.4841,-122.169,CA,None,Menlo Park
4,"{'price_amount': 500000, 'price_currency_code'...","[{'price_amount': None, 'price_currency_code':...",news,Fri May 25 20:03:23 UTC 2007,http://www.crunchbase.com/company/digg,NaN,user driven social content website,2004.0,http://www.digg.com,Digg,...,$45M,135 Mississippi St,,USA,None,37.7647,-122.395,CA,94107,San Francisco
5,"{'price_amount': 39000000, 'price_currency_cod...",[],mobile,Tue Jun 05 21:39:53 UTC 2007,http://www.crunchbase.com/company/helio,NaN,Mobile Virtual Network Operator,2005.0,http://www.helio.com,Helio,...,$0,10960 Wilshire Blvd.,Suite 700,USA,None,34.0575,-118.447,CA,90024,Los Angeles
6,None,[],music,Mon Jun 04 08:45:22 UTC 2007,http://www.crunchbase.com/company/slacker,NaN,Music Personalization Suite,2006.0,http://www.slacker.com,Slacker,...,$73.1M,16935 W. Bernardo Dr. Suite 101,,USA,,33.0222,-117.081,CA,92127,San Diego
7,"{'price_amount': 80000000, 'price_currency_cod...",[],games_video,Tue Jun 05 12:20:29 UTC 2007,http://www.crunchbase.com/company/lala,NaN,hub for online music discovery,NaN,http://www.lala.com,Lala,...,$44.2M,209 Hamilton Ave,Suite #200,USA,Lala Headquarters,37.4512,-122.154,CA,94301,Palo Alto
8,None,[],news,Sat Jun 02 07:34:05 UTC 2007,http://www.crunchbase.com/company/scribd,NaN,Read Unlimited Books,2007.0,http://scribd.com,Scribd,...,$25.8M,539 Bryant Street,,USA,HQ,37.7896,-122.404,CA,94107,San Francisco
9,None,"[{'price_amount': 20000000, 'price_currency_co...",web,Thu May 31 21:46:57 UTC 2007,http://www.crunchbase.com/company/fox-interact...,NaN,,1979.0,http://www.newscorp.com,Fox Interactive Media,...,$0,407 N Maple Dr,,USA,,34.0762,-118.394,CA,90210,Beverly Hills


In [42]:
companies2.shape

(9832, 23)

In [43]:
companies2["geo"] = companies2.apply(lambda x: geopointAir(x["latitude"], x["longitude"]), axis = 1)

In [45]:
companies2.columns

Index(['acquisition', 'acquisitions', 'category_code', 'created_at',
       'crunchbase_url', 'deadpooled_year', 'description_x', 'founded_year',
       'homepage_url', 'name', 'number_of_employees', 'overview', 'products',
       'total_money_raised', 'address1', 'address2', 'country_code',
       'description_y', 'latitude', 'longitude', 'state_code', 'zip_code',
       'city', 'geo'],
      dtype='object')

In [46]:
companiesOffices = json.loads(companies2.T.to_json()).values()
db.companiesOffices.insert_many(companiesOffices)

In [47]:
datatest = db.companiesOffices.find({
    "category_code": {"$in":['software', 'games_video', 'advertising', 'social', 'music', 'travel', 'sports', 'design', 'other']
                     },
})

In [53]:
geopoint = {'type': 'Point', 'coordinates': [-122.4194155, 37.7749295]}
maxdistance = 1000
datatest = db.companiesOffices.find({
        "geo":{
            "$near":{
                "$geometry":geopoint,
                "$maxDistance":maxdistance
            }},
        "category_code": {"$in":['software', 'games_video', 'advertising', 'social', 'music', 'travel', 'sports', 'design', 'other']
                     }})

In [54]:
datatest_df = pd.DataFrame(datatest)

In [58]:
datatest_df

,_id,acquisition,acquisitions,address1,address2,category_code,city,country_code,created_at,crunchbase_url,...,homepage_url,latitude,longitude,name,number_of_employees,overview,products,state_code,total_money_raised,zip_code
0,5d9859e5eb2db0fa7698caa1,None,[],None,None,software,San Francisco,USA,Mon Sep 08 22:16:49 UTC 2008,http://www.crunchbase.com/company/scripped,...,http://scripped.com,37.774929,-122.419415,Scripped,4.0,<p>Scripped provides web-based screenwriting s...,[],CA,$700k,None
1,5d9859e5eb2db0fa7698c8e7,None,[],None,None,software,San Francisco,USA,Mon Sep 08 22:16:49 UTC 2008,http://www.crunchbase.com/company/scripped,...,http://scripped.com,37.774929,-122.419415,Scripped,4.0,<p>Scripped provides web-based screenwriting s...,[],CA,$700k,None
2,5d9859e5eb2db0fa7698bb86,None,[],None,None,social,San Francisco,USA,Sun Dec 09 06:57:44 UTC 2007,http://www.crunchbase.com/company/uversity,...,http://www.uversity.com,37.774929,-122.419415,Uversity,30.0,"<p>Uversity, formerly Inigral, is an industry ...","[{'name': 'Schools App', 'permalink': 'schools...",CA,$10.2M,None
3,5d9859e5eb2db0fa7698c178,None,[],,,other,San Francisco,USA,Thu May 15 21:13:42 UTC 2008,http://www.crunchbase.com/company/venture-hacks,...,http://www.venturehacks.com,37.775196,-122.419204,Venture Hacks,3.0,<p>We&#8217;re Nivi (bio) and Naval (bio). We’...,[],CA,$0,
4,5d9859e5eb2db0fa7698dae5,None,[],,,other,San Francisco,USA,Mon Jun 08 16:44:24 UTC 2009,http://www.crunchbase.com/company/talent-basket,...,http://www.talentbasket.com,37.775196,-122.419204,Talent Basket,3.0,<p>Talent Basket is a human capital agency for...,[],CA,$0,
5,5d9859e5eb2db0fa7698d47d,None,[],,,software,San Francisco,USA,Fri Feb 06 19:54:12 UTC 2009,http://www.crunchbase.com/company/sensor-analy...,...,http://www.sensoranalytics.com,37.775196,-122.419204,Sensor Analytics,NaN,<p>Sensor Analytics helps semiconductor manufa...,[],CA,$0,
6,5d9859e5eb2db0fa7698cf49,None,[],,,software,San Francisco,USA,Mon Dec 01 22:40:50 UTC 2008,http://www.crunchbase.com/company/connectize,...,http://www.connectize.com,37.775196,-122.419204,Connectize,NaN,<p>Connectize: Social Collaboration for Sales ...,"[{'name': 'Connectize Marketplace', 'permalink...",CA,$0,
7,5d9859e5eb2db0fa7698cda7,None,[],,,software,San Francisco,USA,Mon Nov 10 00:23:17 UTC 2008,http://www.crunchbase.com/company/148apps,...,http://148apps.com,37.775196,-122.419204,148Apps,1.0,<p>148Apps is dedicated to bringing the best a...,[],CA,$0,
8,5d9859e5eb2db0fa7698cc14,None,[],,,advertising,San Francisco,USA,Wed Oct 08 06:57:32 UTC 2008,http://www.crunchbase.com/company/adblue,...,http://www.adblueonline.com,37.775196,-122.419204,Adblue,NaN,<p>Adblue is an agency focused on Search Marke...,"[{'name': 'Simulfeed', 'permalink': 'simulfeed...",CA,$0,
9,5d9859e5eb2db0fa7698cbb7,None,[],,,advertising,San Francisco,USA,Wed Oct 01 06:33:33 UTC 2008,http://www.crunchbase.com/company/2020systems,...,http://www.2020systems.com,37.775196,-122.419204,2020systems,NaN,<p>2020systems provides Managed Online Marketi...,[],CA,$0,
